In [5]:
# Import libs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import time
import os
import PIL.Image as Image
from IPython.display import display
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
# Loading dataset
dataset_dir_train = "../input/finalset2/finalDataset"

train_transform = transforms.Compose([transforms.Resize((300, 300)),
                                 transforms.RandomRotation(15),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


dataset = torchvision.datasets.ImageFolder(root=dataset_dir_train, transform = train_transform)
trainloader = torch.utils.data.DataLoader(dataset, batch_size = 32, shuffle=True, num_workers = 2)


In [8]:
# function to train the model
def TrainModel(model, lossFun, optimizer, scheduler, n_epochs=5):
    losses = []
    accuracies = []
    
    model.train()
    
    for epoch in range(n_epochs):
        since = time.time() # record the start time
        running_loss = 0.0
        running_correct = 0.0
        counter = 0
        for i, data in enumerate(trainloader, 0):
            
            inputs, labels = data
            # Asigning GPU
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            counter += len(inputs)
            outputs = model(inputs)
            m = nn.LogSoftmax(dim=1)
            _, predicted = torch.max(outputs.data, 1)
            loss = lossFun(m(outputs), labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            running_correct += (labels==predicted).sum().item()

        timeSpent = time.time()-since # time used
        epoch_loss = running_loss/counter
        print(running_correct, counter)
        epoch_acc = 100*(running_correct/counter)
        print("Epoch %s, Time Used: %d s, loss: %.4f, acc: %.4f" % (epoch+1, timeSpent, epoch_loss, epoch_acc))
        
        losses.append(epoch_loss)
        accuracies.append(epoch_acc)
        scheduler.step(epoch_acc)
    print('Training Finished')
    return model, losses, accuracies

In [9]:
model_dense161 = models.densenet161(pretrained=True)

num_ftrs = model_dense161.classifier.in_features
model_dense161.classifier = nn.Linear(num_ftrs, 23)
model_dense161 = model_dense161.to(device)

lossFun = nn.NLLLoss()
optimizer = optim.SGD(model_dense161.parameters(), lr=0.01, momentum=0.9)
lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold = 0.9)

model_dense161, training_losses, training_accs = TrainModel(model_dense161, lossFun, optimizer, lrscheduler, n_epochs=10)

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth


  0%|          | 0.00/110M [00:00<?, ?B/s]

31758.0 37253
Epoch 1, Time Used: 874 s, loss: 0.0148, acc: 85.2495
35094.0 37253
Epoch 2, Time Used: 867 s, loss: 0.0055, acc: 94.2045
35687.0 37253
Epoch 3, Time Used: 867 s, loss: 0.0039, acc: 95.7963
36094.0 37253
Epoch 4, Time Used: 869 s, loss: 0.0029, acc: 96.8888
36240.0 37253
Epoch 5, Time Used: 872 s, loss: 0.0025, acc: 97.2808
36818.0 37253
Epoch 6, Time Used: 875 s, loss: 0.0011, acc: 98.8323
36940.0 37253
Epoch 7, Time Used: 874 s, loss: 0.0008, acc: 99.1598
36997.0 37253
Epoch 8, Time Used: 876 s, loss: 0.0007, acc: 99.3128
37030.0 37253
Epoch 9, Time Used: 876 s, loss: 0.0006, acc: 99.4014
37053.0 37253
Epoch 10, Time Used: 873 s, loss: 0.0005, acc: 99.4631
Training Finished


In [10]:
model_dense161.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [11]:
m = torch.jit.script(model_dense161)
torch.jit.save(m, 'testing_model.pt')

In [12]:
torch.save(model_dense161, 'model.pt')